<a href="https://colab.research.google.com/github/sean12nokogiri/SQL-order-ID-split/blob/master/mmonthly_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Libraries needed 
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import time
import datetime
import csv
import random
from googleapiclient.errors import HttpError
import socket
from google.colab import files

import calendar

In [18]:
def get_service(api_name, api_version, scopes, key_file_location):
    """Get a service that communicates to a Google API.

    Args:
        api_name: The name of the api to connect to.
        api_version: The api version to connect to.
        scopes: A list auth scopes to authorize for the application.
        key_file_location: The path to a valid service account JSON key file.

    Returns:
        A service that is connected to the specified API.
    """
    
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
            key_file_location, scopes=scopes)

    # Build the service object.
    service = build(api_name, api_version, credentials=credentials)

    return service

In [19]:
def get_results_total(service, profile_id, filt, dte_start, dte_end):
    # Use the Analytics Service Object to query the Core Reporting API
    # for the number of sessions within the past seven days.

    return service.data().ga().get(
            ids='ga:' + profile_id,
            start_date=dte_start,
            end_date=dte_end,
            metrics='ga:users,ga:newUsers,ga:pageviews,ga:timeOnPage,ga:exits,ga:screenviews,ga:timeOnScreen',
            dimensions='ga:deviceCategory,ga:date',
            filters='ga:pagePath=~^' + filt + '*'
            ).execute()


In [20]:
def get_results_irish(service, profile_id, filt, dte_start, dte_end):
    # Use the Analytics Service Object to query the Core Reporting API
    # for the number of sessions within the past seven days.
    return service.data().ga().get(
            ids='ga:' + profile_id,
            start_date=dte_start,
            end_date=dte_end,
            metrics='ga:users,ga:newUsers,ga:pageviews,ga:timeOnPage,ga:exits,ga:screenviews,ga:timeOnScreen',
            dimensions='ga:deviceCategory,ga:yearMonth',
            #filters=';ga:country==Ireland'
            ).execute()

In [21]:
def print_results(results,filt,typee):
    # Open a file.
    filepath = 'monthly_data'     #change this to your actual file path
    filename = 'monthly_data.csv'         #change this to your actual file name
    f = open(filepath + '\\' + filename, 'a')

    # Wrap file with a csv.writer
    writer = csv.writer(f, lineterminator='\n')
   

    #https://www.pythontutorial.net/python-basics/python-write-csv-file/
    # sort out the website subcategories.
    try:
        level1 = filt.split('/')[1]
        level2 = filt.split('/')[2]
    
    except:
        level1 = filt.split('/')[1]
        level2 = 'All'
            
    # Write data table.
    if results.get('rows', []):
        for row in results.get('rows'):
            temp = [typee,level1,level2]
            row = row + temp
            writer.writerow(row)
        
        #print('\n')
        print ('Success Data Written to CSV File for:',filt)
        #print ('filepath = ' + filepath)
        #print ('filename = '+ filename)
      
    else:
        print ('No Rows Found')

    # Close the file.
    f.close() 

In [22]:
#not calling anything beyond the print statement in the makeRequests function. 

def makeRequests():
    filters_list =[ '/News',
                    '/News/Ireland', 
                    '/News/World', 
                    '/News/Politics', 
                    '/News/Crime-and-Law',
                    '/News/Social-Affairs', 
                    '/News/Health', 
                    '/News/Education', 
                    '/Student-Hub/2nd-Level-Student-Hub', 
                    '/News/Consumer',
                    '/News/Environment', 
                    '/News/Science', 
                    '/Sport', 
                    '/Sport/Rugby', 
                    '/Sport/Soccer', 
                    '/Sport/Gaelic-Games', 
                    '/Sport/Golf', 
                    '/Sport/Racing', 
                    '/Sport/Other-Sports', 
                    '/Sport/Women-in-Sport', 
                    '/Business', 
                    '/Business/Economy', 
                    '/Business/Your-Money', 
                    '/Business/Companies', 
                    '/Business/Technology', 
                    '/Business/Financial-Services', 
                    '/Business/Personal-Finance', 
                    '/Business/Work', 
                    '/Business/Health-Pharma', 
                    '/Business/Commercial-Property', 
                    '/Business/Construction', 
                    '/Business/Energy-And-Resources', 
                    '/Business/Media-and-Marketing', 
                    '/Business/Retail-and-services', 
                    '/Business/Transport-and-Tourism', 
                    '/Opinion',
                    '/Opinion/Editorial',
                    '/Opinion/Letters',
                    '/Life-and-Style', 
                    '/Life-and-Style/Food-and-Drink', 
                    '/Life-and-Style/Homes-and-Property', 
                    '/Life-and-Style/Health-Family', 
                    '/Life-and-Style/The-Health-Centre', 
                    '/Life-and-Style/People', 
                    '/Life-and-Style/Travel', 
                    '/Life-and-Style/Motors', 
                    '/Life-and-Style/Fashion', 
                    '/Life-and-Style/Abroad', 
                    '/Culture', 
                    '/Culture/Books', 
                    '/Culture/Film', 
                    '/Culture/Music', 
                    '/Culture/Stage', 
                    '/Culture/Art-and-Design', 
                    '/Culture/TV-Radio-Web', 
                    '/Culture/tuarascáil', 
                    '/Culture/Heritage', 
                    '/Games/Crosswords/']
    scope = 'https://www.googleapis.com/auth/analytics.readonly'
    key_file_location = 'it-paraelle-tracker-0dc6d8a72655.json'


    profile_id = '92770161'
    from_date = '2021-09-01'
    to_date = '2022-05-01'
    #from_date_o = '30daysAgo'
    #to_date_o = 'today'
    from_date_o = datetime.datetime.strptime(from_date,"%Y-%m-%d")
    to_date_o = datetime.datetime.strptime(to_date,"%Y-%m-%d")
    
    # Authenticate and construct service.
    service = get_service(
            api_name='analytics',
            api_version='v3',
            scopes=[scope],
            key_file_location=key_file_location)
    
    t0 = time.time()
    
    # while from_date_o <= to_date_o:
    #     temp_to_date = str(from_date_o.year)+'-'+str(from_date_o.month)+'-'+str(calendar.monthrange(int(from_date_o.year),int(from_date_o.month))[1])
    #     temp_to_date_o = datetime.datetime.strptime(temp_to_date,"%Y-%m-%d")
    #     #irish
    #     for filter_item in filters_list:    
    #         try:
    #             print_results(get_results_irish(service, profile_id,filter_item,str(from_date_o).split(' ')[0],str(temp_to_date_o).split(' ')[0]),filter_item,"Irish")
    #         except socket.timeout:
    #             time.sleep(16)
    #             try:
    #                 print_results(get_results_irish(service, profile_id,filter_item,str(from_date_o).split(' ')[0],str(temp_to_date_o).split(' ')[0]),filter_item,"Irish")
    #             except socket.timeout:
    #                 print("Socket Time out error, code exit")
    #                 exit()
    #     from_date_o += monthdelta()  
    
   
    #total
    for filter_item in filters_list:    
        print_results(get_results_total(service, profile_id,filter_item,str(from_date_o).split(' ')[0],str(to_date_o).split(' ')[0]),filter_item,"Total")
       
    
    t1 = time.time()
    duration = t1-t0
    print(str(time.strftime("%H:%M:%S", time.gmtime(duration))))




In [23]:
def main():

    for n in range(0, 5):
        try:
            makeRequests()
    
        except HttpError as error:
            if error.resp.reason in ['userRateLimitExceeded', 'quotaExceeded',
                                   'internalServerError', 'backendError']:
                
                time.sleep((2 ** n) + random.random())
                print("error number",n)
                
            
            else:
                print(error)
                print("There has been an error, the request never succeeded.")
                break
        
        else:
            break    
if __name__ == '__main__':
    main()


    

Success Data Written to CSV File for: /News
Success Data Written to CSV File for: /News/Ireland
Success Data Written to CSV File for: /News/World
Success Data Written to CSV File for: /News/Politics
Success Data Written to CSV File for: /News/Crime-and-Law
Success Data Written to CSV File for: /News/Social-Affairs
Success Data Written to CSV File for: /News/Health
Success Data Written to CSV File for: /News/Education
Success Data Written to CSV File for: /Student-Hub/2nd-Level-Student-Hub
Success Data Written to CSV File for: /News/Consumer
Success Data Written to CSV File for: /News/Environment
Success Data Written to CSV File for: /News/Science
Success Data Written to CSV File for: /Sport
Success Data Written to CSV File for: /Sport/Rugby
Success Data Written to CSV File for: /Sport/Soccer
Success Data Written to CSV File for: /Sport/Gaelic-Games
Success Data Written to CSV File for: /Sport/Golf
Success Data Written to CSV File for: /Sport/Racing
Success Data Written to CSV File for:

In [24]:
#files.download('monthly_data.csv')